In [40]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [41]:
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt


import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import cut_tree

In [42]:
df= pd.read_csv('/kaggle/input/mcdonalds/mcdonalds.csv')
df.head()

In [43]:
df.info()

In [44]:
num_df=pd.get_dummies(df)
num_df.head()

In [45]:
spec_df=num_df.iloc[:,[0,2,4,6,8,10,12,14,16,18,20,22,40,41]]
spec_df

In [46]:
def func(i):
  if str(i)=='Never':
    return 0
  elif str(i)=='Once a year':
    return 1
  elif str(i)=='Every three months':
    return 2
  elif str(i)=='Once a month':
    return 3
  elif str(i)=='Once a week':
    return 4
  elif str(i)=='More than once a week':
    return 5

In [47]:
list = ['yummy',	'convenient',	'spicy',	'fattening',	'greasy',	'fast',	'cheap'	, 'tasty'	, 'expensive',	'healthy' ,	'disgusting']

for i in list:
  df[i] = df[i].apply(lambda x: 1 if x=="Yes" else 0)

df['Like'] = df['Like'].apply(lambda x:x[-1] if x=="I love it!+5" or "I hate it!-5" else int(x))

df['Like'] = df['Like'].apply(pd.to_numeric)

df['Gender'] = df['Gender'].apply(lambda x:  1 if x=="Male" else 0)

df.describe()

In [48]:
list1 = ['yummy',	'convenient',	'spicy',	'fattening',	'greasy',	'fast',	'cheap'	, 'tasty'	, 'expensive',	'healthy' ,	'disgusting' , 'Gender']

df['VisitFrequency'] = df['VisitFrequency'].astype(str)
vfn = []

for i in df['VisitFrequency']:
  vfn.append(func(i))

df['VisitFrequencynum'] = vfn

x = df.loc[:,list1].values
x

In [49]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
pc = pca.fit_transform(x)
pf = pd.DataFrame(data = pc
             , columns = ['principal component 1', 'principal component 2'])

pf

In [50]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.scatter(pf['principal component 1']
               , pf['principal component 2'])

In [51]:
from sklearn.cluster import KMeans
wcss=[]
for i in range(1,8):
 kmeans = KMeans(i)
 kmeans.fit(x)
 wcss_iter = kmeans.inertia_
 wcss.append(wcss_iter)

number_clusters = range(1,8)
plt.plot(number_clusters,wcss)
plt.title('elbow')
plt.xlabel('Number of clusters')
plt.ylabel('WCSS')

In [52]:
kmeans = KMeans(4)
kmeans.fit(x)

In [53]:
kmeansp = kmeans.predict(x)

labels = kmeans.labels_
print(labels[:100])

In [54]:
df['cluster_num'] = labels
df.head()

In [55]:
plt.scatter(pf['principal component 1'], pf['principal component 2'], c=kmeansp, s=25, cmap='inferno')

In [56]:
from statsmodels.graphics.mosaicplot import mosaic
from itertools import product


for i in list1:
  mosaic(df,[i , 'Like'],title = i)
  plt.show()

In [57]:
m2  = df.groupby(df['cluster_num'])
m2.mean()

In [58]:
m2.max()

In [59]:
m2.min()

In [60]:
plt.figure(figsize=(8,8))
m2.boxplot()

In [61]:
m2.plot()

In [62]:
m2.hist()

In [63]:
m2.Like.mean()

In [64]:
m2.VisitFrequencynum.mean()

In [65]:
plt.scatter(m2.VisitFrequencynum.mean(),m2.Like.mean())

In [66]:
dff = df.loc[df['cluster_num']==2]
dff

In [67]:
dff.describe()

In [68]:
dff2 = dff.drop(['Age','Like','VisitFrequencynum'], axis = 1)
plt.figure(figsize = (30,10))
dff2.boxplot()

In [69]:
dff2.hist()

In [70]:
spec_df.corr()
#Correlation Matrix

plt.figure(figsize=(15,15))
sns.heatmap(spec_df.corr(), annot=True)
plt.show()